In [1]:
import numpy as np
import matplotlib.pyplot as plt

import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'lib')))
from renormalization import rg_step, find_Jc
from utils import build_J

In [ ]:
def compute_full_recursion_matrix(J_full, eps=1e-6):
    """
    Compute the full recursion (Jacobian) matrix for the *actual* RG map.

      J_full (size D)  ->  Jp_full (size D1)
      T_{ij} ≈ d J'_i / d J_j

    Shape:
      D  = len(J_full) - 1
      D1 = len(Jp_full) - 1
      T: (D1 x D)
    """
    D  = len(J_full) - 1
    Jp = rg_step(J_full)
    D1 = len(Jp) - 1

    T = np.zeros((D1, D), dtype=float)

    for j in range(1, D+1):
        J_pert = J_full.copy()
        J_pert[j] += eps
        Jp_pert = full_rg_map(J_pert)

        # assumes same D1 (RG structure fixed by input length)
        diff = (Jp_pert[1:D1+1] - Jp[1:D1+1]) / eps
        T[:, j-1] = diff
    return T, Jp


def newton_rg_lstsq(J_init,
                    max_iter=10,
                    tol=1e-8,
                    eps=1e-6,
                    damping=1.0,
                    verbose=False):
    J = J_init.copy()
    D = len(J) - 1

    for it in range(max_iter):
        T, Jp = compute_full_recursion_matrix(J, eps=eps)
        D1 = len(Jp) - 1

        r = Jp[1:D1+1] - J[1:D1+1]
        res_norm = np.linalg.norm(r)

        if verbose:
            print(f"[lstsq] iter {it}: D={D}, D1={D1}, ||J'-J||={res_norm:.3e}")

        if res_norm < tol:
            return J, {"converged": True, "iterations": it}

        # Solve rectangular system  T δ ≈ r
        delta, *_ = np.linalg.lstsq(T, r, rcond=None)

        J_new = J.copy()
        J_new[1:D+1] = J[1:D+1] - damping * delta

        J = J_new

    return J, {"converged": False, "iterations": max_iter}


def check_fixed_point(J_star, tol=1e-6):
    # physical RG step
    Jp = rg_step(J_star)

    D  = len(J_star) - 1
    D1 = len(Jp) - 1

    # compare only the overlapping region
    diff = Jp[1:D1+1] - J_star[1:D1+1]
    err  = np.linalg.norm(diff)

    print(f"Fixed-point error norm = {err:.3e}")

    # detailed pass/fail
    if err < tol:
        print("✔ Fixed point verified.")
    else:
        print("✘ Not a fixed point (or tolerance too strict).")
    return err

In [ ]:
a = 1
tolerance = 1e-5

Jc = find_Jc(
    a=a,
    Jlow=1e-2, Jhigh=1e2,
    max_steps=6, max_dist_final=6,
    tol=tolerance,
    growth_threshold=1e4, decay_threshold=1e-4
)
print("Critical J =", Jc)

J0 = build_J(J0=Jc, a=a, D=200)

J_star, info = newton_rg_lstsq(J0, verbose=True)
print("Fixed point J*:\n", J_star)

In [ ]:
err = check_fixed_point(J_star, tol=tolerance)